In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import constants as ct

# Limpeza dos dados

Dataset Original

In [2]:
df_gripe = pd.read_csv(ct.PATH_GRIPE, low_memory=False)

Dataset tratado

In [3]:
df_gripe_clean = df_gripe.copy()

Dataset mínimo

In [37]:
df_gripe_min = pd.DataFrame()

Colunas constantes

In [5]:
constant_cols = [x  for x in df_gripe.columns if (len(df_gripe[x].unique())==1)]
constant_cols

['Número da notificação', 'IF - Positivo para outro vírus respiratório']

In [6]:
for c in constant_cols:
    if c in df_gripe_clean.columns:
        del df_gripe_clean[c] 

removed columns

In [7]:
print(len(constant_cols))

2


## Unnamed columns

Removemos as colunas sem nome

In [8]:
unnamed_cols = [x for x in df_gripe.columns if 'unnamed' in x.lower()]

In [9]:
for col in unnamed_cols:
    if col in df_gripe_clean.columns:
        del df_gripe_clean[col]

In [10]:
print(len(unnamed_cols))

16


## 'Data da notificação'

In [11]:
from datetime import datetime

In [12]:
COL_DATA_NOTIF = 'Data da notificação'

Há algumas inconsistências

Assumimos que a coluna é um timestamp e que alguns estão com um 0 a menos

Para as linhas em que data tem menos de 15 0s adicionamos 1, para que os datetimes estejam no mesmo horário

In [13]:
to_fix = np.log10(df_gripe[COL_DATA_NOTIF]) < 15
df_gripe_clean.loc[to_fix,COL_DATA_NOTIF] = df_gripe.loc[to_fix,COL_DATA_NOTIF]*10

In [14]:
np.sum(to_fix)

1843

Convertemos a data em datetime

In [15]:
df_gripe_clean[COL_DATA_NOTIF] = df_gripe_clean[COL_DATA_NOTIF].apply(lambda x:
                            datetime.fromtimestamp(x/1e6))

Range de Datas de Notificação

In [16]:
df_gripe_clean[COL_DATA_NOTIF].describe()

count                   17545
unique                     92
top       2020-05-07 22:06:40
freq                     1163
first     2020-01-02 18:20:00
last      2020-05-14 20:46:40
Name: Data da notificação, dtype: object

## 'Semana epidemiológica da notificação'

Por convenção internacional as semanas epidemiológicas são contadas de domingo a sábado.

In [17]:
COL_DATA_SEM_EP = "Semana epidemiológica da notificação"

Convertemos a data em datetime

In [18]:
df_gripe_clean[COL_DATA_SEM_EP] = pd.to_datetime(
    df_gripe[COL_DATA_SEM_EP], format='%d/%m/%Y')

Descrição das datas

In [19]:
df_gripe_clean[COL_DATA_SEM_EP].describe()

count                   17545
unique                    120
top       2020-04-27 00:00:00
freq                      695
first     2020-01-01 00:00:00
last      2020-05-14 00:00:00
Name: Semana epidemiológica da notificação, dtype: object

In [20]:
future_sem = df_gripe_clean[COL_DATA_SEM_EP] > df_gripe_clean[COL_DATA_NOTIF]

Alguns dos casos a semana epidemiológica da notificação vem depois da data de notificação.

Mas a diferença é de um dia geralmente

Exceto por um caso em que esse valor é maior que 14 dias. O que não faz sentido

In [21]:
diff_notif_sem = df_gripe_clean[future_sem][COL_DATA_SEM_EP]- df_gripe_clean[future_sem][COL_DATA_NOTIF]
diff_notif_sem

134      0 days 01:53:20
3954     0 days 08:06:40
4849     0 days 01:53:20
5689     0 days 00:33:20
6318     0 days 01:53:20
6987     0 days 01:53:20
7981     0 days 00:20:00
8167     0 days 01:40:00
8465     0 days 00:20:00
8576     0 days 00:20:00
11324    0 days 08:06:40
11518    0 days 01:26:40
12293    0 days 01:53:20
12819    0 days 01:53:20
13152    0 days 01:26:40
13674    0 days 01:53:20
13888    0 days 04:20:00
14303    0 days 04:20:00
15450    0 days 04:06:40
16306    0 days 01:40:00
16628    0 days 01:40:00
16737    0 days 01:40:00
17037    0 days 00:33:20
17158    0 days 04:06:40
17181   14 days 06:33:20
dtype: timedelta64[ns]

In [22]:
diff_notif_sem.max()

Timedelta('14 days 06:33:20')

Removemos a data com maior discrepância

In [23]:
removed_row = diff_notif_sem.index[diff_notif_sem.argmax()]

In [24]:
if removed_row in df_gripe_clean.index:
    df_gripe_clean = df_gripe_clean.drop(removed_row)

## 'Data de nascimento'

Acho que deram o nome errado a coluna

In [25]:
COL_NASCIMENTO_WRONG = 'Data de nascimento'
df_gripe_clean[COL_NASCIMENTO_WRONG].unique()

array(['1 - Masculino', '2 - Feminino', '9 - Ignorado'], dtype=object)

In [26]:
df_gripe_clean[COL_NASCIMENTO_WRONG].value_counts()

1 - Masculino    9641
2 - Feminino     7884
9 - Ignorado       19
Name: Data de nascimento, dtype: int64

Renomeamos a coluna

In [27]:
COL_SEXO = '_sexo'
df_gripe_clean[COL_SEXO] = df_gripe_clean[COL_NASCIMENTO_WRONG] 

## 'Sexo'

Difícil saber do que se trata realmente, parece ser um código

In [28]:
COL_SEXO_WRONG = 'Sexo'
df_gripe_clean[COL_SEXO_WRONG].value_counts()

3034984    501
2298120    415
2696851    387
3005992    341
5717256    329
          ... 
12548        1
2288230      1
6539858      1
27014        1
6003494      1
Name: Sexo, Length: 447, dtype: int64

Removemos a coluna 

In [29]:
if COL_SEXO_WRONG in df_gripe_clean:
    del df_gripe_clean[COL_SEXO_WRONG]

## 'Município notificante'

A vasta maioria dos casos são do rio de janeiro neste dataset

In [30]:
COL_MUN_NOTIF = 'Município notificante'
df_gripe[COL_MUN_NOTIF].value_counts()

RJ    17475
SP       30
MG       20
PR        3
BA        2
ES        2
AL        2
SC        2
DF        1
GO        1
SE        1
RS        1
PA        1
MT        1
MA        1
PI        1
CE        1
Name: Município notificante, dtype: int64

Filtramos apenas os casos do rio de janeiro e excluímos a coluna

In [31]:
df_gripe_clean = df_gripe_clean[df_gripe_clean[COL_MUN_NOTIF]=='RJ']

In [32]:
if COL_MUN_NOTIF in df_gripe_clean:
    del df_gripe_clean[COL_MUN_NOTIF]

## 'Unidade da idade'

Parece ser a verdadeira coluna para data de nascimento

In [33]:
COL_UNIDADE_IDADE = 'Unidade da idade'
COL_DATA_NASCIMENTO = 'Data de Nascimento'
df_gripe['Unidade da idade']

0        16/04/1953
1        14/11/1960
2        26/12/1936
3        22/07/1978
4        05/11/1945
            ...    
17540    12/12/1950
17541    27/01/1951
17542    04/09/1979
17543    13/06/1943
17544    01/01/1939
Name: Unidade da idade, Length: 17545, dtype: object

## 'Unidade notificante'

Aparentemente OK

In [35]:
COL_UNID = 'Unidade notificante'
df_gripe_clean[COL_UNID]

0        330455
1        330455
2        330455
3        330580
4        330455
          ...  
17540    330490
17541    330455
17542    330240
17543    330455
17544    330455
Name: Unidade notificante, Length: 17474, dtype: int64

In [38]:
df_gripe_min[COL_UNID] = df_gripe_clean[COL_UNID]
df_gripe_min[COL_DATA_NOTIF] = df_gripe_clean[COL_DATA_NOTIF]

## 'CNES da unidade notificante'

Acredito que seja o nome fantasia na verdade

In [43]:
COL_CNES_WRONG = 'CNES da unidade notificante'
COL_NO_FANTASIA = 'NO_FANTASIA'
df_gripe_clean[COL_NO_FANTASIA] = df_gripe_clean[COL_CNES_WRONG]
if COL_CNES_WRONG in df_gripe_clean.columns:
    del df_gripe_clean[COL_CNES_WRONG]

## 'Gestante'

Acredito que a coluna é a de idade

In [48]:
COL_GESTANTE_WRONG = 'Gestante'
COL_IDADE = 'Idade'

In [50]:
df_gripe_clean[COL_IDADE] = df_gripe_clean[COL_GESTANTE_WRONG]
if COL_GESTANTE_WRONG in df_gripe_clean.columns:
    del df_gripe_clean[COL_GESTANTE_WRONG]

In [51]:
df_gripe_min[COL_IDADE] = df_gripe_clean[COL_IDADE]

## 'ZONA'

Creio que seja bairro

In [60]:
COL_ZONA_WRONG = 'Zona'
COL_BAIRRO = '_Bairro'

In [62]:
df_gripe_clean[COL_BAIRRO] = df_gripe_clean[COL_ZONA_WRONG]

In [64]:
df_gripe_min[COL_BAIRRO] = df_gripe_clean[COL_BAIRRO]

## 'Houve internação  para SRAG'

In [74]:
COL_INTERN_SRAG_WRONG = 'Houve internação para SRAG'

Creio que é a data de internação, mas tipos mistos

In [75]:
df_gripe_clean[COL_INTERN_SRAG_WRONG].unique()

array([nan, '25/04/2020', '02/05/2020', '28/04/2020', '05/03/2020',
       '16/03/2020', '03/03/2020', '10/04/2020', '30/04/2020',
       '10/05/2020', '03/05/2020', '13/04/2020', '26/04/2020',
       '29/03/2020', '25/03/2020', '18/04/2020', '29/04/2020',
       '18/03/2020', '04/04/2020', '06/04/2020', '20/04/2020', '2 - Não',
       '24/04/2020', '21/03/2020', '23/04/2020', '06/05/2020', '1 - Sim',
       '28/03/2020', '14/04/2020', '17/03/2020', '21/01/2020',
       '27/04/2020', '01/05/2020', '17/04/2020', '16/04/2020',
       '05/04/2020', '04/05/2020', '19/04/2020', '03/04/2020',
       '05/05/2020', '12/05/2020', '09/04/2020', '23/03/2020',
       '15/04/2020', '12/04/2020', '09/03/2020', '01/04/2020',
       '22/04/2020', '12/03/2020', '02/04/2020', '08/04/2020',
       '09/05/2020', '11/04/2020', '24/03/2020', '30/03/2020',
       '07/05/2020', '09/01/2020', '04/02/2020', '02/03/2020',
       '08/05/2020', '10/01/2020', '21/04/2020', '26/03/2020',
       '07/04/2020', '13/03/

In [76]:
COL_CNES_INTERNACAO = 'CNES da unidade de saúde de internação'

Quase toas da mesma unidade

In [86]:
df_gripe_clean[COL_NO_FANTASIA].value_counts()

HOSPITAL QUINTA DOR                               501
SMS HOSPITAL MUNICIPAL ALBERT SCHWEITZER AP 51    415
HOSPITAL SAO GONCALO LTDA                         387
HOSPITAL COPA DOR                                 341
SMS HOSPITAL MUNICIPAL RONALDO GAZOLLA AP 33      329
                                                 ... 
SMS RIO HOSPITAL MATERNIDADE ALEXANDER FLEMING      1
SAMCI HOSPITAL INFANTIL                             1
UPA UNIDADE DE PRONTO ATENDIMENTO BARRA             1
LABORATORIO ARCILIO GUIMARAES                       1
PRONTO ATENDIMENTO INFANTIL                         1
Name: NO_FANTASIA, Length: 379, dtype: int64

In [82]:
df_gripe_min.sample(3)

,Unidade notificante,Data da notificação,Idade,_Bairro
17444,330455,2020-03-31 21:13:20,54,VILA DA PENHA
6240,330455,2020-05-06 18:20:00,88,AUSTIN
12579,330390,2020-05-14 20:46:40,73,VILA ITAMARATI


## 'Internado em UTI'

In [99]:
COL_INTERNADO_UTI_WRONG = 'Internado em UTI'
COL_CNES_INTERNADO = 'CNES internado'

In [101]:
df_gripe_min[COL_CNES_INTERNADO] = df_gripe_clean[COL_INTERNADO_UTI_WRONG]

In [108]:
del df_gripe_min[COL_UNID]

In [113]:
from imp import reload
reload(ct)

<module 'constants' from '/home/rafael/git/HackCovid_CaminhoEmergencia/notebooks/inspecao_dados/constants.py'>

In [117]:
df_gripe_min = df_gripe_min[~df_gripe_min['CNES internado'].isna()]

In [122]:
df_gripe_min = df_gripe_min.rename({'_Bairro':'Bairro'},axis=1)

In [123]:
df_gripe_min.to_csv(ct.PATH_GRIPE_MIN, index=False)

In [115]:
df_gripe_min.shape

(17474, 4)

In [79]:
COL_UF_INTERNACAO_WRONG = 'UF de internação'
COL_DATA_INTERNACAO = 'Data de internação'
df_gripe[COL_UF_INTERNACAO_WRONG].unique()

array(['29/04/2020', '03/05/2020', '21/04/2020', nan, '25/04/2020',
       '30/04/2020', '27/03/2020', '01/05/2020', '28/04/2020',
       '08/04/2020', '15/04/2020', '05/05/2020', '18/04/2020',
       '11/05/2020', '02/04/2020', '12/04/2020', '07/05/2020',
       '03/03/2020', '09/04/2020', '16/03/2020', '27/04/2020',
       '04/05/2020', '02/03/2020', '17/04/2020', '01/04/2020',
       '03/04/2020', '06/05/2020', '10/05/2020', '19/03/2020',
       '04/04/2020', '13/04/2020', '06/04/2020', '11/04/2020',
       '08/05/2020', '16/04/2020', '28/03/2020', '29/03/2020',
       '20/04/2020', '19/04/2020', '25/03/2020', '26/04/2020',
       '13/05/2020', '05/04/2020', '09/05/2020', '17/03/2020',
       '10/04/2020', '18/03/2020', '31/03/2020', '14/04/2020',
       '07/04/2020', '21/03/2020', '30/03/2020', '22/04/2020',
       '23/04/2020', '24/04/2020', '13/03/2020', '23/03/2020',
       '20/03/2020', '12/05/2020', '21/01/2020', '22/03/2020',
       '02/05/2020', '26/03/2020', '24/03/2020', '

In [97]:
df_gripe['Internado em UTI'].nunique()

441

In [96]:
df_gripe['Internado em UTI'].value_counts()

3034984    395
330455     377
2298120    376
2696851    375
5717256    322
          ... 
2269945      1
6629989      1
330200       1
2089610      1
9684948      1
Name: Internado em UTI, Length: 441, dtype: int64

In [124]:
df_gripe_min

,Data da notificação,Idade,Bairro,CNES internado
0,2020-05-13 17:00:00,67,BONSUCESSO,2270234
1,2020-05-07 22:06:40,59,JARDIM METROPOLE,6716911
2,2020-04-27 12:06:40,83,SANTA CRUZ,2270692
4,2020-04-29 19:40:00,74,PACIENCIA,6598544
5,2020-05-12 13:13:20,68,CENTRO,6681573
...,...,...,...,...
17537,2020-04-18 05:53:20,78,MARE,7107366
17539,2020-04-28 15:53:20,72,BANGU,2298120
17541,2020-04-14 18:33:20,69,BARRA DA TIJUCA,9065946
17543,2020-03-29 13:40:00,76,JARDIM CARIOCA,5717256


In [98]:
df_gripe[COL_CNES_INTERNACAO].nunique()

484

In [55]:
list(df_gripe.columns)

['Número da notificação',
 'Data da notificação',
 'Semana epidemiológica da notificação',
 'Data dos primeiros sintomas',
 'Semana epidemiológica dos primeiros sintomas',
 'UF notificante',
 'Município notificante',
 'Código do município notificante',
 'Unidade notificante',
 'CNES da unidade notificante',
 'Sexo',
 'Data de nascimento',
 'Unidade da idade',
 'Idade',
 'Gestante',
 'Raça/cor',
 'Etnia',
 'Escolaridade',
 'Ocupação',
 'CBO da ocupação',
 'UF de residência',
 'Município de residência',
 'Código do município de residência',
 'Bairro',
 'Zona',
 'País de residência',
 'Código do país de residência',
 'Histórico de viagem internacional',
 'País viagem internacional',
 'Código país viagem internacional',
 'Local viagem internacional',
 'Data de partida viagem internacional',
 'Data de retorno viagem internacional',
 'Caso de surto de SG que evoluiu para SRAG',
 'Caso nosocomial - adquirido no hospital',
 'Contato com aves ou suínos',
 'Sintomas - febre',
 'Sintomas - tosse'

'IF - Positivo para

In [54]:
df_gripe.columns[df_gripe.isna().sum(axis=0)/len(df_gripe) > 0.9]

Index(['Escolaridade', 'CBO da ocupação', 'UF de residência',
       'Código país viagem internacional', 'Local viagem internacional',
       'Data de partida viagem internacional',
       'Data de retorno viagem internacional',
       'Caso de surto de SG que evoluiu para SRAG',
       'Fatores de risco - tem outros', '< 6 Meses: Mãe recebeu vacina',
       'Data da vacinação da mãe', '< 6 Meses: Mãe amamenta',
       'Data da vacinação dose única', 'Data da vacinação 1ª dose',
       'Data da vacinação 2ª dose', 'Usou antiviral para gripe',
       'Antiviral - todos', 'Uso de suporte ventilatório',
       'IF - Positivo para outros vírus',
       'IF - Positivo para vírus sincicial respiratório',
       'IF - Positivo para parainfluenza 1',
       'IF - Positivo para parainfluenza 2',
       'IF - Positivo para parainfluenza 3', 'IF - Positivo para adenovírus',
       'IF - Outro vírus respiratórios', 'IF - Todos os resultados',
       'RT-PCR - Outro subtipo da influenza A',
       

In [24]:
df_gripe['Unidade notificante'].unique()

array([330455, 330580, 330330, 330452, 330170, 330020, 330550, 330070,
       330200, 330620, 330350, 330285, 330520, 330490, 330610, 330390,
       330240, 221100, 330510, 330050, 330010, 330022, 330030, 330340,
       330430, 330411, 330630, 330100, 330470, 330190, 330575, 330110,
       355030, 330420, 530010, 330093, 330400, 330187, 330475, 330415,
       330220, 330210, 330515, 330130, 330600, 330080, 330270, 330320,
       330500, 330290, 330250, 330360, 330045, 330040, 330395, 314390,
       316370, 330540, 330185, 330060, 311340, 330090, 330280, 310150,
       355410, 330023, 330180, 330205, 330380, 292740, 150270, 330015,
       350390, 230440, 520620, 291920, 330370, 320120, 270430, 310620,
       316250, 330460, 330440, 354990, 313240, 510340, 420820, 312510,
       316720, 313670, 211130, 431490, 330225, 354850, 351340, 330150,
       410840, 351880, 355220, 354980, 350950, 351380, 420200, 320530,
       410420, 330310, 280030])

In [30]:
df_gripe['Semana epidemiológica da notificação'] = pd.to_datetime(df_gripe['Semana epidemiológica da notificação'])

In [31]:
df_gripe['Semana epidemiológica da notificação'].min()

Timestamp('2020-01-01 00:00:00')

In [32]:
df_gripe['Semana epidemiológica da notificação'].max()

Timestamp('2020-12-05 00:00:00')

In [5]:
df_gripe.sample(3)

,Número da notificação,Data da notificação,Semana epidemiológica da notificação,Data dos primeiros sintomas,Semana epidemiológica dos primeiros sintomas,UF notificante,Município notificante,Código do município notificante,Unidade notificante,CNES da unidade notificante,...,Unnamed: 152,Unnamed: 153,Unnamed: 154,Unnamed: 155,Unnamed: 156,Unnamed: 157,Unnamed: 158,Unnamed: 159,Unnamed: 160,Unnamed: 161
13817,3,1586900000000000,12/04/2020,16,07/04/2020,15,RJ,CABO FRIO,330070,HOSPITAL SANTA IZABEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9907,3,1587800000000000,14/04/2020,16,11/04/2020,15,RJ,SAO GONCALO,330490,HOSPITAL SAO GONCALO LTDA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14816,3,1588900000000000,28/04/2020,18,27/04/2020,18,RJ,RIO DE JANEIRO,330455,UNIMED RIO PA BARRA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [126]:
df_gripe_min['Idade']

0        67
1        59
2        83
4        74
5        68
         ..
17537    78
17539    72
17541    69
17543    76
17544    81
Name: Idade, Length: 13905, dtype: int64

In [10]:
df_gripe.isna().sum(axis=0)

Número da notificação                               0
Data da notificação                                 0
Semana epidemiológica da notificação                0
Data dos primeiros sintomas                         0
Semana epidemiológica dos primeiros sintomas        0
                                                ...  
Unnamed: 157                                    17542
Unnamed: 158                                    17544
Unnamed: 159                                    17544
Unnamed: 160                                    17544
Unnamed: 161                                    17544
Length: 162, dtype: int64